In [5]:
from helper import *
from datasetcreator import *
from dependencies import *
from loaddata import review_data,business_data, rest_data, user_data, elite_udata, non_elite_udata


User Data Loaded
Businesses Data Loaded
Review Data Loaded


In [6]:
cv, bid_2_stars_ne, review_data_ne, bid_2_textenc = give_text_feats_tfidf(review_data)
bid_2_stars_e = review_data.loc[review_data['iselite'] == True].groupby('business_id')['stars'].apply(list).to_dict()

Size of Vocabulary:  15374


In [32]:
def create_dataset_multiclass(min_thold, bdata, bid_2_stars, bid_2_stars_e, bid_2_textenc, max_thold=float('inf')):
    basicprint = False
    x1 = []
    x2 = []
    y = []
    bids = bdata['business_id'].values
    for bid in bids:
        if bid in bid_2_stars and bid in bid_2_stars_e and bid in bid_2_textenc:
            stars_e = bid_2_stars_e[bid]
            stars_ne = bid_2_stars[bid]
            lestars = len(stars_e)
            if(lestars >= min_thold and lestars < max_thold):
                x = []
                x.append([np.mean(stars_ne)])
                countratings_ne = [Counter(stars_ne)[i] for i in range(1,6)]
                distribution_ne = [x/sum(countratings_ne) for x in countratings_ne]
                x.append(distribution_ne)
                x.append(give_categ(np.mean(stars_ne)))
                x.append(bid_2_textenc[bid])
                x = [item for sublist in x for item in sublist]
                x1.append(x)
                countratings_e = [Counter(stars_e)[i] for i in range(1,6)]
                distribution_e = [x/sum(countratings_e) for x in countratings_e]
                y.append(distribution_e)                        
    xfeat = np.array(x1)
    imp = SimpleImputer(missing_values=np.nan, strategy='mean')
    xfeat = imp.fit_transform(xfeat)
    yfeat = np.array(y)
    print("Dimension of Our Dataset: ", xfeat.shape)
    return xfeat,yfeat

In [57]:
xfeat,yfeat = create_dataset_multiclass(40,business_data,bid_2_stars_ne,bid_2_stars_e,bid_2_textenc)
xtrain, xtest, ytrain, ytest = give_train_test_multi(xfeat, yfeat)
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# scaler.fit(xtrain) 
# xtrain = scaler.transform(xtrain)  
# xtest = scaler.transform(xtest)

Dimension of Our Dataset:  (5926, 15421)


In [84]:
def create_dataset_multiclass(min_thold, bdata, bid_2_stars, bid_2_stars_e, bid_2_textenc, max_thold=float('inf')):
    basicprint = False
    x1 = []
    x2 = []
    y = []
    bids = bdata['business_id'].values
    for bid in bids:
        if bid in bid_2_stars and bid in bid_2_stars_e and bid in bid_2_textenc:
            stars_e = bid_2_stars_e[bid]
            stars_ne = bid_2_stars[bid]
            lestars = len(stars_e)
            if(lestars >= min_thold and lestars < max_thold):
                x = []
                x.append([np.mean(stars_ne)])
                countratings_ne = [Counter(stars_ne)[i] for i in range(1,6)]
                distribution_ne = [x/sum(countratings_ne) for x in countratings_ne]
                x.append(distribution_ne)
                x.append(give_categ(np.mean(stars_ne)))
                x.append(bid_2_textenc[bid])
                x = [item for sublist in x for item in sublist]
                x1.append(x)
                countratings_e = [Counter(stars_e)[i] for i in range(1,6)]
                distribution_e = [x/sum(countratings_e) for x in countratings_e]
                y.append(distribution_e)                        
    xfeat = np.array(x1)
    imp = SimpleImputer(missing_values=np.nan, strategy='mean')
    xfeat = imp.fit_transform(xfeat)
    yfeat = np.array(y)
    print("Dimension of Our Dataset: ", xfeat.shape)
    return xfeat,yfeat

In [90]:
from tensorflow.keras.layers import Dense
import tensorflow as tf

model = tf.keras.Sequential()
model.add(Dense(50, input_dim=15421, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(5, activation='softmax'))
opt = tf.keras.optimizers.SGD(lr=0.01, momentum=0.9)
model.compile(loss='kullback_leibler_divergence', optimizer=opt, metrics=['accuracy'])



In [91]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
xtrain_scaled = scaler.fit_transform(xtrain)
xtest_scaled = scaler.transform(xtest)
history = model.fit(xtrain, ytrain, epochs = 100)

Train on 4740 samples
Epoch 1/100
4740/4740 [==============================] - 1s 310us/sample - loss: 0.1194 - accuracy: 0.6196
Epoch 2/100
4740/4740 [==============================] - 1s 197us/sample - loss: 0.0797 - accuracy: 0.7013
Epoch 3/100
4740/4740 [==============================] - 1s 196us/sample - loss: 0.0659 - accuracy: 0.7291
Epoch 4/100
4740/4740 [==============================] - 1s 199us/sample - loss: 0.0607 - accuracy: 0.7346
Epoch 5/100
4740/4740 [==============================] - 1s 198us/sample - loss: 0.0582 - accuracy: 0.7496
Epoch 6/100
4740/4740 [==============================] - 1s 203us/sample - loss: 0.0565 - accuracy: 0.7549
Epoch 7/100
4740/4740 [==============================] - 1s 216us/sample - loss: 0.0556 - accuracy: 0.7574
Epoch 8/100
4740/4740 [==============================] - 1s 222us/sample - loss: 0.0545 - accuracy: 0.7624
Epoch 9/100
4740/4740 [==============================] - 1s 221us/sample - loss: 0.0538 - accuracy: 0.7624
Epoch 10/100
47

In [92]:
ytestpred = model.predict(xtest)

In [93]:
y1 = [x[0]*1+x[1]*2+x[2]*3+x[3]*4+x[4]*5 for x in ytestpred] 
y2 = [x[0]*1+x[1]*2+x[2]*3+x[3]*4+x[4]*5 for x in ytest] 

In [94]:
print_prediction(np.array(y1),np.array(y2))

MSE:  0.027
R-Squared :  0.8465
Margin  0.10  0.20  0.30  0.40  0.50
        49.16  79.43  93.59  97.47  99.16
 


0.8464532623070217

In [95]:
ytestpred

array([[0.01220664, 0.03433712, 0.12517762, 0.44543287, 0.38284573],
       [0.02541594, 0.07397583, 0.18636912, 0.4056808 , 0.3085583 ],
       [0.01212482, 0.04916609, 0.17726123, 0.45290557, 0.30854228],
       ...,
       [0.01336025, 0.03853995, 0.14366622, 0.4707299 , 0.3337036 ],
       [0.0133213 , 0.04974233, 0.18091749, 0.42772612, 0.3282928 ],
       [0.00818162, 0.02923386, 0.13595237, 0.48767376, 0.3389584 ]],
      dtype=float32)